In [ ]:
import arcpy
import os

# выберем слой
aprx = arcpy.mp.ArcGISProject('current') # текущий проект
m = aprx.listMaps('Lesson 2')[0] # карта
lyr = m.listLayers('soil_samples_with_analysis')[0] # слой точек опробования

# считаем все столбцы с слое точек опробования
fields = [i.name for i in arcpy.ListFields(lyr)]
for i, field in enumerate(fields):
    print(i, fields[i])

In [ ]:
# сделаем выборку слоёв
fields_subset = fields[5:41]

In [ ]:
for field in fields_subset:
    # напишем название для нового слоя
    lyr_out_name = field.replace('AR_MS', 'AR-MS').replace('pct', '%')
    lyr_out_name = lyr_out_name.split('_')[0] +\
                    ' (' + lyr_out_name.split('_')[1] +\
                    ', ' + lyr_out_name.split('_')[2] + ')'

    # создадим новый слой
    lyr_out = arcpy.management.MakeFeatureLayer(lyr, lyr_out_name)[0]

    # покажем точки в градуированной шкале
    sym = lyr_out.symbology
    sym.updateRenderer('GraduatedSymbolsRenderer')
    sym.renderer.classificationField = field
    sym.renderer.breakCount = 5 # количество классов
    sym.renderer.maximumSymbolSize = 14 # максимальный размер точки
    sym.renderer.minimumSymbolSize = 6 # минимальный размер точки

    # зададим цветовую шкалу
    # выбрать цветовую шкалу, например: 'Bathymetric Scale', либо создать свою
    sym.renderer.colorRamp = aprx.listColorRamps('5 level black to red')[0] 

    # update symbology
    lyr_out.symbology = sym

    # получим фактическое количество классов
    sym = lyr_out.symbology
    number_of_classes = sym.renderer.breakCount
    print(f'Количество фактических классов: {number_of_classes}')

    # изменим форматирование
    cim = lyr_out.getDefinition('V3')

    # сразу отключим слой
    cim.visibility = False

    # зададим формат легенды
    cim.renderer.numberFormat = {
              "type": "CIMNumericFormat",
              "alignmentOption": "esriAlignLeft",
              "alignmentWidth": 0,
              "roundingOption": "esriRoundNumberOfSignificantDigits",
              "roundingValue": 3,
              "zeroPad": True
            }

    try:
        # заменим подписи
        label_class_1 = [i for i in cim.labelClasses if i.name == 'Class 1'][0]

        # получим пограничное значение второго высшего класса
        upper_bound = cim.renderer.breaks[-2].upperBound

        # составим выражение для подписи
        label_class_1.expression = "if($feature.{0} > {1}){2}return $feature.{3}{4}".format(
                                                            field, upper_bound, '{', field,'}')
        label_class_1.expressionEngine = "Arcade"

        # зададим свойства текста
        label_class_1.textSymbol.symbol.fontFamilyName = 'Arial'
        label_class_1.textSymbol.symbol = {
                      "type": "CIMTextSymbol",
                      "blockProgression": "TTB",
                      "depth3D": 0,
                      "extrapolateBaselines": True,
                      "flipAngle": 90,
                      "fontEffects": "Normal",
                      "fontEncoding": "Unicode",
                      "fontFamilyName": "Arial",
                      "fontStyleName": "Regular",
                      "fontType": "TTOpenType",
                      "haloSize": 1,
                      "haloSymbol": {
                        "type": "CIMPolygonSymbol",
                        "symbolLayers": [
                          {
                            "type": "CIMSolidFill",
                            "enable": True,
                            "color": {
                              "type": "CIMRGBColor",
                              "values": [
                                104,
                                104,
                                104,
                                74.90196078431373
                              ]
                            }
                          }
                        ],
                        "angleAlignment": "Map"
                      },
                      "height": 10,
                      "hinting": "Default",
                      "horizontalAlignment": "Left",
                      "kerning": True,
                      "letterSpacing": 9,
                      "letterWidth": 100,
                      "ligatures": False,
                      "lineGapType": "ExtraLeading",
                      "symbol": {
                        "type": "CIMPolygonSymbol",
                        "symbolLayers": [
                          {
                            "type": "CIMSolidFill",
                            "enable": True,
                            "color": {
                              "type": "CIMRGBColor",
                              "values": [
                                255,
                                190,
                                190,
                                100
                              ]
                            }
                          }
                        ],
                        "angleAlignment": "Map"
                      },
                      "symbol3DProperties": {
                        "type": "CIM3DSymbolProperties",
                        "dominantSizeAxis3D": "Z",
                        "rotationOrder3D": "XYZ",
                        "scaleZ": 1,
                        "scaleY": 1
                      },
                      "textCase": "Normal",
                      "textDirection": "LTR",
                      "verticalAlignment": "Baseline",
                      "verticalGlyphOrientation": "Right",
                      "wordSpacing": 100,
                      "billboardMode3D": "FaceNearPlane"
                    }
        label_class_1.useCodedValue = True
        label_class_1.visibility = True

        # обновим свойства слоя
        cim.labelClasses = [label_class_1]
        cim.labelVisibility = True
    except:
        print(f"Обновление формата подписей и графики для слоя {field} не было удачным")
    
    # Обновим свойства слоя
    lyr_out.setDefinition(cim)
    
    # сохраним lyrx файл и импортируем, чтобы "исправить баг ESRI", 
    # когда не обновляется формат чисел в легенде
    lyrx_file = field + '.lyrx'
    arcpy.management.SaveToLayerFile(lyr_out, lyrx_file, 'RELATIVE')
    arcpy.management.ApplySymbologyFromLayer(in_layer = lyr_out,
                                            in_symbology_layer = lyrx_file,
                                            symbology_fields = f"VALUE_FIELD {field} {field}",
                                            update_symbology = "UPDATE")